# SMA Crossover Strategy
Author: TraderPy

Youtube: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ/featured

Website: https://traderpy.com/

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
import MetaTrader5 as mt5
mt5.initialize("C:\\Program Files\\MetaTrader 5 IC Markets (SC)\\terminal64.exe")
login = 51317684
password1 ='mdmYhgTT'
server = 'ICMarketsSC-Demo'
mt5.login(login,password1,server)

True

In [3]:
# settings
symbol = 'USDCAD'
timeframe = mt5.TIMEFRAME_D1
start_pos = 0
num_bars = 1000

fsma_period = 10
ssma_period = 100

In [4]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df['fast_sma'] = df['close'].rolling(fsma_period).mean()
df['slow_sma'] = df['close'].rolling(ssma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df.dropna(inplace=True)
df

,time,open,high,low,close,fast_sma,slow_sma,prev_fast_sma
99,2020-07-31,1.34220,1.34379,1.33722,1.34101,1.341347,1.382070,1.343060
100,2020-08-03,1.34004,1.34512,1.33801,1.33910,1.339921,1.381450,1.341347
101,2020-08-04,1.33886,1.34205,1.33187,1.33194,1.338521,1.380570,1.339921
102,2020-08-05,1.33183,1.33302,1.32336,1.32633,1.336971,1.379316,1.338521
103,2020-08-06,1.32659,1.33232,1.32438,1.33106,1.336011,1.378111,1.336971
...,...,...,...,...,...,...,...,...
995,2024-01-15,1.34038,1.34483,1.33820,1.34265,1.337270,1.357499,1.335563
996,2024-01-16,1.34215,1.35021,1.34215,1.34945,1.339006,1.357390,1.337270
997,2024-01-17,1.34808,1.35420,1.34803,1.35047,1.340529,1.357297,1.339006
998,2024-01-18,1.35023,1.35270,1.34807,1.34871,1.341908,1.357233,1.340529


In [32]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

signal = df[df['crossover'] == 'bullish crossover'].copy()
signal

,time,open,high,low,close,fast_sma,slow_sma,prev_fast_sma,crossover
457,2021-11-22,1.26398,1.27052,1.26282,1.27007,1.256756,1.255586,1.254159,bullish crossover
507,2022-01-31,1.27643,1.27771,1.26820,1.27070,1.262644,1.262281,1.260742,bullish crossover
570,2022-04-28,1.28164,1.28798,1.27910,1.28067,1.268142,1.267970,1.266129,bullish crossover
605,2022-06-16,1.28882,1.29703,1.28606,1.29470,1.274290,1.271416,1.270518,bullish crossover
787,2023-02-28,1.35750,1.36483,1.35608,1.36454,1.352406,1.350880,1.349312,bullish crossover
831,2023-05-01,1.35334,1.35832,1.35302,1.35435,1.353467,1.352569,1.351965,bullish crossover
849,2023-05-25,1.35938,1.36449,1.35859,1.36433,1.352009,1.350852,1.350468,bullish crossover
906,2023-08-14,1.34335,1.34800,1.34323,1.34606,1.339295,1.338468,1.336575,bullish crossover


In [38]:
#define levels
l1 = 1.36000
l2 = 1.35550
l3 = 1.34950

#visualization of the levels

In [33]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [34]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
            
            if data.crossover == 'bullish crossover':
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [35]:
sma_crossover_strategy = Strategy(df, 10000, 1)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-11-22,1.27007,buy,1,0,0,2022-01-17,1.25168,-0.01839,closed,9999.98161
1,2022-01-31,1.27070,buy,1,0,0,2022-03-23,1.25616,-0.01454,closed,9999.96707
2,2022-04-28,1.28067,buy,1,0,0,2022-06-06,1.25790,-0.02277,closed,9999.94430
3,2022-06-16,1.29470,buy,1,0,0,2023-04-07,1.35073,0.05603,closed,10000.00033
4,2023-02-28,1.36454,buy,1,0,0,2023-04-07,1.35073,-0.01381,closed,9999.98652
5,2023-05-01,1.35435,buy,1,0,0,2023-05-10,1.33710,-0.01725,closed,9999.96927
6,2023-05-25,1.36433,buy,1,0,0,2023-06-07,1.33707,-0.02726,closed,9999.94201
7,2023-08-14,1.34606,buy,1,0,0,NaT,NaN,NaN,open,NaN


In [36]:
px.line(result, x='close_datetime', y='pnl')

In [37]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in result[result['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

In [40]:
import plotly.graph_objects as go

# Create a sample scatter plot
trace = go.Scatter(x=[1, 2, 3, 4], y=[10, 11, 12, 13], mode='markers', name='Data')

# Create a layout with a horizontal line at y=12
layout = go.Layout(shapes=[{
    'type': 'line',
    'x0': 0,
    'x1': 1,
    'xref': 'paper',
    'y0': 12,
    'y1': 12,
    'yref': 'y',
    'line': {'color': 'red','width': 2,'dash': 'dashdot',}
}])

# Create the figure and add the trace and layout
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()